# A/B test 支付宝效果分析

本分析以支付宝营销活动为例，通过广告点击率指标比较两组营销策略的广告投放效果

## 一、查看数据

本测试主要使用到effect_tb.csv文件

文件有2645957行，4列数据，包含对照组以及2个测试组的所有数据

## 二、导入库

In [2]:
import numpy as np
import matplotlib as plt
import pandas as pd

## 三、读取数据

In [4]:
df=pd.read_csv(r'F:\Data\ab test\effect_tb.csv')

## 四、查看数据

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2645957 entries, 0 to 2645956
Data columns (total 4 columns):
 #   Column  Dtype
---  ------  -----
 0   1       int64
 1   1.1     int64
 2   0       int64
 3   1.2     int64
dtypes: int64(4)
memory usage: 80.7 MB


In [6]:
df.head()

,1,1.1,0,1.2
0,1,1000004,0,1
1,1,1000004,0,2
2,1,1000006,0,1
3,1,1000006,0,3
4,1,1000007,0,1


In [7]:
df.describe()

,1,1.1,0,1.2
count,2.645957e+06,2.645957e+06,2.645957e+06,2.645957e+06
mean,1.440575e+00,3.112996e+06,1.456297e-02,1.395762e+00
std,4.964562e-01,1.828261e+06,1.197952e-01,6.920481e-01
min,1.000000e+00,1.100000e+01,0.000000e+00,1.000000e+00
25%,1.000000e+00,1.526777e+06,0.000000e+00,1.000000e+00
50%,1.000000e+00,3.062185e+06,0.000000e+00,1.000000e+00
75%,2.000000e+00,4.721132e+06,0.000000e+00,2.000000e+00
max,2.000000e+00,6.265402e+06,1.000000e+00,3.000000e+00


## 五、数据清洗

### 修改列名

4列数据的含义分别是：
- dt: 日志天数
- user_id: 用户Id
- label: 判断用户是否点击，0表示没点击，1表示点击
- dmp_id: 营销策略编号(1: 对照组, 2: 营销策略1, 3: 营销策略2)

本例中，日志天数不需要使用，可以直接删除本列

In [8]:
df.columns=['dt','user_id','label','dmp_id']

In [9]:
df.drop('dt',axis=1,inplace=True)

In [10]:
# 查看数据
df.head()

,user_id,label,dmp_id
0,1000004,0,1
1,1000004,0,2
2,1000006,0,1
3,1000006,0,3
4,1000007,0,1


In [11]:
df.describe()

,user_id,label,dmp_id
count,2.645957e+06,2.645957e+06,2.645957e+06
mean,3.112996e+06,1.456297e-02,1.395762e+00
std,1.828261e+06,1.197952e-01,6.920481e-01
min,1.100000e+01,0.000000e+00,1.000000e+00
25%,1.526777e+06,0.000000e+00,1.000000e+00
50%,3.062185e+06,0.000000e+00,1.000000e+00
75%,4.721132e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


### 取出重复值

In [13]:
# 查看重复值
df.shape

(2645957, 3)

In [19]:
df.nunique()

user_id    2410682
label            2
dmp_id           3
dtype: int64

我们发现一共2645957行数据中，只有2410682个user_id,说明有重复id，需要进行筛选

In [21]:
# 查看全部重复数据
df[df.user_id.duplicated(keep=False)].sort_values(by='user_id')

,user_id,label,dmp_id
1766238,24,0,2
1766237,24,0,1
1065566,51,0,3
1065565,51,0,1
2585795,68,0,3
...,...,...,...
2575238,6265375,0,3
2575237,6265375,0,2
2575236,6265375,0,1
1382444,6265385,0,2


In [22]:
df=df.drop_duplicates()

In [24]:
# 再次查看重复值
df.duplicated().sum()

0

In [25]:
df.isnull().sum()

user_id    0
label      0
dmp_id     0
dtype: int64

In [27]:
# 查看数据
df.describe()

,user_id,label,dmp_id
count,2.632974e+06,2.632974e+06,2.632974e+06
mean,3.112327e+06,1.462073e-02,1.396326e+00
std,1.828223e+06,1.200290e-01,6.924164e-01
min,1.100000e+01,0.000000e+00,1.000000e+00
25%,1.526414e+06,0.000000e+00,1.000000e+00
50%,3.061416e+06,0.000000e+00,1.000000e+00
75%,4.720377e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


In [28]:
# 查看数据类型
df.dtypes

user_id    int64
label      int64
dmp_id     int64
dtype: object

In [30]:
# 查看1,2,3组数据量
df.dmp_id.value_counts()

1    1905662
2     411107
3     316205
Name: dmp_id, dtype: int64

## 六、保存数据

In [32]:
df.to_csv(r'F:\Data\ab test\zfb.csv')

## 七、假设检验

### 数据处理

In [33]:
per1=df[(df.dmp_id==1)&(df.label==1)].shape[0]/df[df.dmp_id==1].shape[0]
per2=df[(df.dmp_id==2)&(df.label==1)].shape[0]/df[df.dmp_id==2].shape[0]
per3=df[(df.dmp_id==3)&(df.label==1)].shape[0]/df[df.dmp_id==3].shape[0]

In [39]:
# 查看各组实验的转化率
print("对照组 ：%.20f" %per1)
print("实验组1：%.20f" %per2)
print("实验组2：%.20f" %per3)

对照组 ：0.01255101901596400622
实验组1：0.01531474774207201503
实验组2：0.02619186919877927386


我们可以看出，实验组2提高了1.3个百分点，而实验组1只提高了0.3个百分点，我们选取实验2进行假设检验

进行假设检验的步骤请参考我上一个A/B test实验，其中有详细的说明，在这里不赘述

### 零假设和备择假设

对照组的点击率为per1，实验组2的点击率为per3

零假设H<sub>0</sub>：per1>=per3

备择假设H<sub>1</sub>：per1<per3

### 检验方向

选用左尾为检验方向

### 选择检验类型以及检验统计量

由于检验量较大(大于30)，选用z检验进行检验

### 显著性水平α

这里选用α为0.05

### 实验组2的python计算

#### 导入库

In [43]:
import statsmodels.stats.proportion as sp

#### 确定数据

In [49]:
con_old=df[(df.dmp_id==1)&(df.label==1)].shape[0]
con_old

23918

In [50]:
con_new=df[(df.dmp_id==3)&(df.label==1)].shape[0]
con_new

8282

In [47]:
n_old=df[df.dmp_id==1].shape[0]
n_old

1905662

In [48]:
n_new=df[df.dmp_id==3].shape[0]
n_new

316205

这里我们的per1<per2，使用参数'smaller'

In [51]:
z_score, p_value = sp.proportions_ztest([con_old, con_new], [n_old, n_new], alternative='smaller')

#### 查看参数

In [52]:
z_score

-59.44164223047762

In [53]:
p_value

0.0

由于p_value=0.0<0.05=α，所以我们认为发生H<sub>0</sub>的概率比较小，我们拒绝H<sub>0</sub>，接受H<sub>1</sub>

### 实验组1的python计算

#### 导入库

In [54]:
import statsmodels.stats.proportion as sp

#### 确定数据

In [62]:
con_old=df[(df.dmp_id==1)&(df.label==1)].shape[0]
con_old

23918

In [63]:
con_new=df[(df.dmp_id==2)&(df.label==1)].shape[0]
con_new

6296

In [64]:
n_old=df[df.dmp_id==1].shape[0]
n_old

1905662

In [65]:
n_new=df[df.dmp_id==2].shape[0]
n_new

411107

这里我们的per1<per2，使用参数'smaller'

In [66]:
z_score, p_value = sp.proportions_ztest([con_old, con_new], [n_old, n_new], alternative='smaller')

#### 查看参数

In [67]:
z_score

-14.1658361296907

In [68]:
p_value

7.454093047070575e-46

由于p_value<0.05=α，所以我们认为发生H<sub>0</sub>的概率比较小，我们拒绝H<sub>0</sub>，接受H<sub>1</sub>

## 八、结论

- 两种营销策略对点击量都有明显的提升
- 策略一提升了0.3个百分点，策略二提升了1.3个百分点
- 两种策略选取策略二进行实施受益最大